# Model

VDSR\
epoch : 80\
filter : 32\
layers : 5\
loss function : L1

In [1]:
import os
import torch.nn as nn
import torch.optim as optim
from torchvision.transforms import *
import torch
import math 
from os.path import join
import torch.utils.data as data
import numpy as np
from os import listdir
from os.path import join
from PIL import Image, ImageOps
import random
from random import randrange
from math import sqrt
       
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------        
class Conv_ReLU_Block(nn.Module): #used for VDSR
    def __init__(self):
        super(Conv_ReLU_Block, self).__init__() #super를 안해주면 부모 클래스의 init 함수가 Overriding 된다.
        self.conv = nn.Conv2d(in_channels=32, out_channels = 32, kernel_size = 3, stride = 1, padding = 1, bias =False)
        self.relu = nn.ReLU(inplace=True) #inplace 연산은 결과값을 새로운 변수에 값을 저장하는 대신 기존의 데이터를 대체하는것
    def forward(self, x):
        return self.relu(self.conv(x)) 
    
class VDSR(nn.Module):
    def __init__(self):
        super(VDSR, self).__init__()
        self.input = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False)
        self.residual_layer = self.make_layer(Conv_ReLU_Block, 5)
        self.output = nn.Conv2d(in_channels=32, out_channels=3, kernel_size=3, stride=1, padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        
        for m in self.modules(): #모델의 모듈을 불러온다
            if isinstance(m, nn.Conv2d): #만약 그 모듈이 Conv2d이면 초기화를 해준다.
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, sqrt(2. / n)) #weight initialization (평균, 표준편차)
    def make_layer(self, block, num_of_layer):
        layers=[]
        for _ in range(num_of_layer):
            layers.append(block())
        return nn.Sequential(*layers) #*layers -> unpacking(deal with list elements separately)
    def forward(self, x):
        residual = x
        out = self.relu(self.input(x))
        out = self.residual_layer(out)
        out = self.output(out)
        out = torch.add(out, residual)
        return out
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------  
def is_image_file(filename): #image file 인지 확인해줌
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg",".bmp"])
    
def load_img(filepath):
    img = Image.open(filepath).convert('RGB')
    return img
    
def rescale_img(img_in, scale):
    size_in = img_in.size
    new_size_in = tuple([int(x * scale) for x in size_in])
    img_in = img_in.resize(new_size_in, resample = Image.BICUBIC)
    return img_in
    
def get_patch(img_in, img_tar, img_bic, patch_size, scale, ix= -1, iy = -1): 
    #잘 이해 안됨. 각 크기가 맞는 랜덤한 패치를 return 하는거?
    (ih, iw) = img_in.size
    (th, tw) = (scale * ih, scale * iw)
    
    patch_mult = scale
    tp = patch_mult * patch_size
    ip = tp // scale
    
    if ix == -1:
        ix = random.randrange(0, iw - ip + 1)
    if iy == -1:
        iy = random.randrange(0, ih - ip + 1)

    (tx, ty) = (scale * ix, scale * iy)

    img_in = img_in.crop((iy,ix,iy + ip, ix + ip))
    img_tar = img_tar.crop((ty,tx,ty + tp, tx + tp))
    img_bic = img_bic.crop((ty,tx,ty + tp, tx + tp))
                
    info_patch = {
        'ix': ix, 'iy': iy, 'ip': ip, 'tx': tx, 'ty': ty, 'tp': tp}

    return img_in, img_tar, img_bic, info_patch

def augment(img_in, img_tar, img_bic, flip_h=True, rot=True):
    info_aug = {'flip_h': False, 'flip_v': False, 'trans': False}
    
    if random.random() < 0.5 and flip_h :
        img_in = ImageOps.flip(img_in)
        img_tar = ImageOps.flip(img_tar)
        img_bic = ImageOps.flip(img_bic)
        info_aug['flip_h'] = True
    if rot : 
        if random.random() < 0.5:
            img_in = ImageOps.mirror(img_in)
            img_tar = ImageOps.mirror(img_tar)
            img_bic = ImageOps.mirror(img_bic)
            info_aug['flip_v'] = True
        if random.random() < 0.5:
            img_in = img_in.rotate(180)
            img_tar = img_tar.rotate(180)
            img_bic = img_bic.rotate(180)
            info_aug['trans'] = True
    return img_in, img_tar, img_bic, info_aug
    
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
class DatasetFromFolder(data.Dataset): #training dataset class
    def __init__(self, image_dir, patch_size, upscale_factor, data_augmentation, transform=None):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]  #image file들의 name(dir)을 담은 리스트
        self.patch_size = patch_size
        self.upscale_factor = upscale_factor
        self.transform = transform
        self.data_augmentation = data_augmentation
    def __getitem__(self, index):
        target = load_img(self.image_filenames[index])
        
        input = target.resize((int(target.size[0]/self.upscale_factor), int(target.size[1]/self.upscale_factor)), Image.BICUBIC) 
        bicubic = rescale_img(input, self.upscale_factor) #upscale_factor로 줄이고 다시 늘려서 LR image 를 만든것 같다.(방식은 bicubic)
        input, target, bicubic, _ = get_patch(input,target,bicubic,self.patch_size, self.upscale_factor) #crop된 image로 다시 정의함
        
        if self.data_augmentation :
            input, target, bicubic, _ = augment(input, target, bicubic)
        if self.transform :
            input = self.transform(input)
            bicubic = self.transform(bicubic)
            target = self.transform(target)
            
        return input, target, bicubic #그럼 최종 return은 patch(crop된 image)인 건가?
    
    def __len__(self):
        return len(self.image_filenames)
    
class DatasetFromFolderEval(data.Dataset): #test dataset class
    
    def __init__(self, lr_dir, upscale_factor, transform = None):
        super(DatasetFromFolderEval, self).__init__()
        self.image_filenames = [join(lr_dir, x) for x in listdir(lr_dir) if is_image_file(x)]
        self.upscale_factor = upscale_factor
        self.transform = transform
        
    def __getitem__(self, index):
        target = load_img(self.image_filenames[index])
        _, file = os.path.split(self.image_filenames[index])  #주소 split해서 파일이름을 가지는 file 변수 정의(head, tail) tail은 마지막 구성요소
        
        input = target.resize((int(target.size[0]/self.upscale_factor),int(target.size[1]/self.upscale_factor)), Image.BICUBIC)
        bicubic = rescale_img(input, self.upscale_factor)
        
        if self.transform:
            input = self.transform(input)
            bicubic = self.transform(bicubic)
            target = self.transform(target)
        
        return input, bicubic, target, file
        
    def __len__(self):
        return len(self.image_filenames)
        
def transform():
    return Compose([
        ToTensor(),
    ])
def get_training_set(data_dir, hr, upscale_factor, patch_size, data_augmentation):
    hr_dir = join(data_dir, hr)
    return DatasetFromFolder(hr_dir, patch_size, upscale_factor, data_augmentation, 
                            transform = transform())
def get_eval_set(lr_dir, upscale_factor):
    return DatasetFromFolderEval(lr_dir, upscale_factor,
                                transform = transform())

# Training

In [ ]:
from __future__ import print_function
import argparse
from math import log10

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pdb
import socket
import time
import easydict

from google.colab import drive
drive.mount('/content/gdrive')

opt  = easydict.EasyDict({ 
    "batchSize": 32,           # batch size - 한번에 training할 patch의 숫자
    "lr": 1e-3,                # learning rate
    "upscale_factor": 2,       # Upscale 정도 - 2배, 4배
    "patch_size": 64,         # patch 크기
    "feature_number": 64,     # SRCNN model에서 사용할 feature 숫자
    
    "start_iter": 1,           
    "nEpochs": 80,            # training 횟수
    "snapshots": 20,           # weight 저장 주기
   
    "data_dir":"/content/gdrive/My Drive/study/VDSR/data/train/", # dataset 저장 위치
    "hr_train_dataset": "DIV2K_train_HR", # training에 사용할 dataset 종류

    "model_type": "VDSR",         # 모델이름

    "save_folder": "/content/gdrive/My Drive/study/VDSR/weights/", # weight 저장 위치
    

    "pretrained_sr": "/content/gdrive/My Drive/study/VDSR/weights/ckpt_diff11_label",
    "pretrained": False,
    "data_augmentation": False,
    "residual": False,
    "gpu_mode": True,
    "threads": 1,
    "seed": 123,
    "gpus": 1,
    "prefix": "tpami_residual_filter8"})

gpus_list = range(opt.gpus)
hostname = str(socket.gethostname) #저장 파일명으로 사용하려고 그냥 가져온것
cudnn.benchmark = True # cuDNN이 최적의 알고리즘을 선택하도록 해줌(잘 이해 안됨)
print(opt)

def train(epoch):
    epoch_loss = 0
    model.train()
    for iteration, batch in enumerate(training_data_loader, 1): #시작 인덱스(i)를 1부터 세겠다는 뜻
        _, target, bicubic = Variable(batch[0]), Variable(batch[1]), Variable(batch[2]) #torch.autograd.Variable 하는 이유??
        if cuda : 
             # input = input.cuda(gpus_list[0])
            target = target.cuda(gpus_list[0])
            bicubic = bicubic.cuda(gpus_list[0])
        optimizer.zero_grad()
        t0 = time.time()
        prediction = model(bicubic)
        
        if opt.residual :
            prediction = prediction + bicubic
        
        loss = criterion(prediction, target)
        t1 = time.time()
        epoch_loss += loss.data
        loss.backward()
        optimizer.step()
        
        print("===> Epoch[{}]({}/{}) : Loss: {:.4f} || Timer: {:.4f} sec.".format(epoch, iteration, len(training_data_loader), loss.data, (t1-t0))) 
    print("===> Epoch {} Complete : Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))
 #-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------       
def print_network(net):  #어디에 쓰임????
    num_params = 0
    for param in net.parameters():
        num_params += param.numel()
    print(net)
    print('Total number of parameters: %d' % num_params)
    
def checkpoint(epoch): #check point 마다 weight를 저장해줌 (모델의 매개변수들을 저장해준다)
    model_out_path = opt.save_folder+hostname+"_epoch_{}.pth".format(epoch)
    torch.save(model.state_dict(), model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
if __name__ == '__main__':
    
    cuda = opt.gpu_mode
    if cuda and not torch.cuda.is_available():
        raise Exception("No GPU found, please run without --cuda")
    torch.manual_seed(opt.seed)
    if cuda : 
        torch.cuda.manual_seed(opt.seed)
    
    print('===> Loading datasets')
    train_set = get_training_set(opt.data_dir, opt.hr_train_dataset, opt.upscale_factor, opt.patch_size, opt.data_augmentation)
    training_data_loader = DataLoader(dataset = train_set, num_workers=opt.threads, batch_size = opt.batchSize, shuffle=True)
    
    print('===> Building model ', opt.model_type)
    if opt.model_type == 'VDSR':
        model = VDSR()
        model = torch.nn.DataParallel(model, device_ids=gpus_list)
    criterion = nn.L1Loss()   #Loss function
    
    if opt.pretrained:
        model_name = os.path.join(opt.save_folder + opt.pretrained_sr)
        if os.path.exists(model_name):
            #model= torch.load(model_name, map_location=lambda storage, loc: storage)
            model.load_state_dict(torch.load(model_name, map_location=lambda storage, loc: storage))
            print('Pre-trained SR model is loaded.')  
    if cuda:
        model = model.cuda(gpus_list[0])
        criterion = criterion.cuda(gpus_list[0])
        
    optimizer = optim.Adam(model.parameters(), lr = opt.lr, betas = (0.9, 0.999), eps=1e-8)
    
    for epoch in range(opt.start_iter, opt.nEpochs +1):
        train(epoch)
        #learning rate decay
        if (epoch+1) % (opt.nEpochs/2) == 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] /= 10.0
            print('Learning rate decay : lr={}'.format(optimizer.param_groups[0]['lr']))
        
        if (epoch+1) % (opt.snapshots) == 0:
            checkpoint(epoch) 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
{'batchSize': 32, 'lr': 0.001, 'upscale_factor': 2, 'patch_size': 64, 'feature_number': 64, 'start_iter': 1, 'nEpochs': 80, 'snapshots': 20, 'data_dir': '/content/gdrive/My Drive/study/SRCNN/data/train/', 'hr_train_dataset': 'DIV2K_train_HR', 'model_type': 'VDSR', 'save_folder': '/content/gdrive/My Drive/study/SRCNN/weights/', 'pretrained_sr': '/content/gdrive/My Drive/study/SRCNN/weights/ckpt_diff11_label', 'pretrained': False, 'data_augmentation': False, 'residual': False, 'gpu_mode': True, 'threads': 1, 'seed': 123, 'gpus': 1, 'prefix': 'tpami_residual_filter8'}
===> Loading datasets
===> Building model  VDSR
===> Epoch[1](1/25) : Loss: 0.2987 || Timer: 0.7361 sec.
===> Epoch[1](2/25) : Loss: 0.1969 || Timer: 0.0050 sec.
===> Epoch[1](3/25) : Loss: 0.0813 || Timer: 0.0043 sec.
===> Epoch[1](4/25) : Loss: 0.0628 || Timer: 0.0044 sec.


KeyboardInterrupt: ignored

# Testing

In [3]:
from __future__ import print_function
import argparse

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from functools import reduce
from math import log10

# from scipy.misc import imsave
import scipy.io as sio
import time
import cv2
import easydict
from google.colab import drive
drive.mount('/content/gdrive')

opt = easydict.EasyDict({ 
    "testBatchSize": 1, 
    "self_ensemble": False,
    "chop_forward": False,
    "gpu_mode": True,
    "threads": 1,
    "seed": 123,
    "gpus": 1,
    "upscale_factor": 2,

    "input_dir":"/content/gdrive/My Drive/study/VDSR/data/test/",  # test dataset 불러올위치
    "test_dataset": "Set5",                             # 사용할 test dataset

    "output": "/content/gdrive/My Drive/study/VDSR/Results/", # 결과영상 저장위치

    "model_type": "VDSR",
    "residual": False,
    "model": "/content/gdrive/My Drive/study/VDSR/weights/test03/<built-in function gethostname>_epoch_79.pth"})  # test에 사용할 weight 불러올 위치 ac040091dff2_epoch_59.pth
#<built-in function gethostname>_epoch_79.pth

gpus_list = range(opt.gpus)
print(opt)

cuda = opt.gpu_mode
if cuda and not torch.cuda.is_available():
    raise Exception("No GPU found, please run without --cuda")
    
torch.manual_seed(opt.seed)
criterion = nn.MSELoss()



print('===> Loading datasets')
test_set = get_eval_set(os.path.join(opt.input_dir,opt.test_dataset), opt.upscale_factor)
testing_data_loader = DataLoader(dataset=test_set, num_workers=opt.threads, batch_size=opt.testBatchSize, shuffle=False)
print('===> Building model')
model = VDSR()
model = torch.nn.DataParallel(model, device_ids = gpus_list)
model.load_state_dict(torch.load(opt.model, map_location=lambda storage, loc : storage))
print('Pre-trained SR model is loaded.')

if cuda:
    model = model.cuda(gpus_list[0])
    criterion = criterion.cuda(gpus_list[0])
    

    
def eval():
    avg_psnr = 0
    psnr_sq = 0
    model.eval()
    for batch in testing_data_loader:
        with torch.no_grad():
            input, bicubic, target,name = Variable(batch[0]), Variable(batch[1]), Variable(batch[2]), batch[3]
        if cuda : 
            input = input.cuda(gpus_list[0])
            bicubic = bicubic.cuda(gpus_list[0])
            target = target.cuda(gpus_list[0])
            
        t0 = time.time()
        if opt.chop_forward : 
            with torch.no_grad():
                prediction = chop_forward(input, model, opt.upscale_factor)
        else : 
            if opt.self_ensemble :
                with torch.no_grad():
                    prediction = x8_forward(input, model)
            else : 
                with torch.no_grad():    
                    prediction = model(bicubic)  #현재 모델의 경우 이를 수행
        if opt.residual : 
            prediction = prediction + bicubic
        
        mse = criterion(prediction, target)
        psnr = 10 * log10(1 / mse.item())
        avg_psnr += psnr / len(testing_data_loader)
        psnr_sq += psnr*psnr
    
        t1 = time.time()
        print("===> Processing: %s || Timer: %.4f sec. PSNR : %.4f" % (name[0], (t1 - t0), psnr))
        
        save_img(bicubic.cpu().data, "bicubic_"+name[0])
        save_img(prediction.cpu().data, "prediction_"+name[0])
        save_img(target.cpu().data, "target_"+name[0])
        
    variance = psnr_sq / len(testing_data_loader) - avg_psnr*avg_psnr
    print("===> Processing Done, Average PSNR : %.4f" % (avg_psnr))
    print("===>PSNR Variance : %.4f" % (variance))
    
def save_img(img, img_name):
    save_img = img.squeeze().clamp(0, 1).numpy().transpose(1,2,0)
    # save img
    save_dir=os.path.join(opt.output,opt.test_dataset)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    save_fn = save_dir +'/'+ img_name
    cv2.imwrite(save_fn, cv2.cvtColor(save_img*255, cv2.COLOR_BGR2RGB),  [cv2.IMWRITE_PNG_COMPRESSION, 0])

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------
def x8_forward(img, model, precision = 'single'):  #if self_ensemble = True
    def _transform(v, op):
        if precision != 'single' : 
            v=v.float()
        v2np = v.data.cpu().numpy()  #np array로 바꿔줌
        if op == 'vflip' :
            tfnp = v2np[:, :, :, ::-1].copy()
        elif op == 'hflip':
            tfnp = v2np[:, :, ::-1, :].copy()
        elif op == 'transpose':
            tfnp = v2np.transpose((0, 1, 3, 2)).copy()
        
        ret = torch.Tensor(tfnp).cuda() #다시 Tensor로 변환
        
        if precision == 'half': #float16
            ret = ret.half()
        elif precision == 'double': #float64
            ret = ret.double()

        with torch.no_grad():
            ret = Variable(ret)

        return ret
            
    inputlist = [img]
    for tf in 'vflip', 'hflip', 'transpose': #inputlist에 'vflip', 'hflip', 'transpose' 시킨 data를 추가해줌 (augmentation?)
        inputlist.extend([_transform(t, tf) for t in inputlist])
        
    outputlist = [model(aug) for aug in inputlist] #늘어난 inputlist를 model에 넣어준 결과를 outputlist에 저장
    for i in range(len(outputlist)): #outputlist의 값들도 augmentation 하는건가???
        if i > 3:
            outputlist[i] = _transform(outputlist[i], 'transpose')
        if i % 4 > 1:
            outputlist[i] = _transform(outputlist[i], 'hflip')
        if (i % 4) % 2 == 1:
            outputlist[i] = _transform(outputlist[i], 'vflip')
    output = reduce((lambda x, y : x+y), outputlist) / len(outputlist)   #outputlist안에 data 누적해서 더하고 평균 구함
    return output
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------
def chop_forward(x, model, scale, shave=8, min_size=80000, nGPUs=opt.gpus): #if chop_forward = True
    b, c, h, w = x.size()
    h_half, w_half = h//2, w//2
    h_size, w_size = h_half + shave, w_half + shave   #chop 하는 size
    inputlist = [
        x[:, :, 0:h_size, 0:w_size],                       #4 방향 chop
        x[:, :, 0:h_size, (w - w_size):w],
        x[:, :, (h - h_size):h, 0:w_size],
        x[:, :, (h - h_size):h, (w - w_size):w]
    ]
    if w_size * h_size < min_size:
        outputlist = []
        for i in range(0, 4, nGPUs):
            with torch.no_grad():
                input_batch = torch.cat(inputlist[i:(i + nGPUs)], dim=0)
            if opt.self_ensemble:
                with torch.no_grad():
                    output_batch = x8_forward(input_batch, model)
            else:
                with torch.no_grad():
                    output_batch = model(input_batch)
            outputlist.extend(output_batch.chunk(nGPUs, dim=0))
    else : 
        outputlist = [
            chop_forward(patch, model, scale, shave, min_size, nGPUs)
            for patch in inputlist
        ] #회귀형 구조로 min_size 보다 작을때까지 진행시켜주며 outputlist에 추가시켜준다
        
    h, w = scale * h, scale * w
    h_half, w_half = scale * h_half, scale * w_half
    h_size, w_size = scale * h_size, scale * w_size
    shave *= scale
    
    with torch.no_grad():
        output = Variable(x.data.new(b, c, h, w))
#outputlist에 저장된 이미지들을 upscale 시킨 크기로 다시 chop해서 output에 저장(처음 x가 downscale된 상태로 들어옴)
    output[:, :, 0:h_half, 0:w_half] \
        = outputlist[0][:, :, 0:h_half, 0:w_half]
    output[:, :, 0:h_half, w_half:w] \
        = outputlist[1][:, :, 0:h_half, (w_size - w + w_half):w_size]
    output[:, :, h_half:h, 0:w_half] \
        = outputlist[2][:, :, (h_size - h + h_half):h_size, 0:w_half]
    output[:, :, h_half:h, w_half:w] \
        = outputlist[3][:, :, (h_size - h + h_half):h_size, (w_size - w + w_half):w_size]

    return output
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------    
    
if __name__ == '__main__':
    eval()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
{'testBatchSize': 1, 'self_ensemble': False, 'chop_forward': False, 'gpu_mode': True, 'threads': 1, 'seed': 123, 'gpus': 1, 'upscale_factor': 2, 'input_dir': '/content/gdrive/My Drive/study/VDSR/data/test/', 'test_dataset': 'Set5', 'output': '/content/gdrive/My Drive/study/VDSR/Results/', 'model_type': 'VDSR', 'residual': False, 'model': '/content/gdrive/My Drive/study/VDSR/weights/test03/<built-in function gethostname>_epoch_79.pth'}
===> Loading datasets
===> Building model
Pre-trained SR model is loaded.
===> Processing: woman_GT.png || Timer: 0.0054 sec. PSNR : 33.6457
===> Processing: baby_GT.png || Timer: 0.0132 sec. PSNR : 36.8398
===> Processing: bird_GT.png || Timer: 0.0047 sec. PSNR : 37.5207
===> Processing: head_GT.png || Timer: 0.0045 sec. PSNR : 32.0398
===> Processing: butterfly_GT.png || Timer: 0.0037 sec. PSNR : 30.6040
===> Processing: Fin